<a href="https://colab.research.google.com/github/KinshukTr/ChatBotMemory/blob/main/Chatbotmemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_google_genai
!pip install -q langchain-groq groq
!pip install -q transformers sentence-transformers



In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "ChatBot_With_Memory"

In [42]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from langchain_groq import ChatGroq  # Import Groq's chat model
model = ChatGroq(model="llama-3.3-70b-versatile")

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [7]:
print(model.invoke("Ola!!").content)

Olá. How can I help you today?


In [8]:
import bs4
from langchain import hub

In [9]:
from langchain.chains import create_retrieval_chain

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [11]:
from langchain_chroma import Chroma

In [12]:
from langchain_community.document_loaders import WebBaseLoader

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
from langchain_core.prompts import MessagesPlaceholder


In [16]:
loader = WebBaseLoader(web_paths=("https://ai.stanford.edu/blog/introduction-to-knowledge-graphs/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)



In [17]:
doc = loader.load()


doc

[Document(metadata={'source': 'https://ai.stanford.edu/blog/introduction-to-knowledge-graphs/'}, page_content='\nKnowledge Graphs (KGs) have emerged as a compelling abstraction for organizing the world’s structured knowledge, and as a way to integrate information extracted from multiple data sources. Knowledge graphs have started to play a central role in representing the information extracted using natural language processing and computer vision. Domain knowledge expressed in KGs is being input into machine learning models to produce better predictions. Our goals in this blog post are to (a) explain the basic terminology, concepts, and usage of KGs, (b) highlight recent applications of KGs that have led to a surge in their popularity, and (c) situate KGs in the overall landscape of AI. This blog post is a good starting point before reading a more extensive survey or following research seminars on this topic.\nKnowledge Graph Definition\nA directed labeled graph is a 4-tuple G = (N, E,

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [19]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()


retriever


VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7afad6962550>, search_kwargs={})

In [20]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [21]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)


rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [23]:
response = rag_chain.invoke({"input":"What are nodes in knowledge graphs?"})


response["answer"]

'In knowledge graphs, nodes can be anything, such as people, companies, computers, or objects, and they represent entities. These nodes are connected by edges that capture relationships between them. Nodes can also represent classes of objects, such as Book or Textbook, in a taxonomy.'

In [24]:
from langchain.chains import create_history_aware_retriever

In [25]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "Formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [26]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)


In [27]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, qa_prompt)


rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

In [29]:
chat_history = []


question1 = "what is a 360 degree view?"

In [30]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})


message1["answer"]

'A 360-degree view refers to a comprehensive and integrated view of a customer that combines external information about the customer with internal information about the same customer. This view integrates data from various sources, such as publicly available information, commercially sourced data, and internal customer information, to provide a complete picture of the customer. It is often represented using a knowledge graph (KG) that tracks key entities, events, and relationships.'

In [31]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)


chat_history

[HumanMessage(content='what is a 360 degree view?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='A 360-degree view refers to a comprehensive and integrated view of a customer that combines external information about the customer with internal information about the same customer. This view integrates data from various sources, such as publicly available information, commercially sourced data, and internal customer information, to provide a complete picture of the customer. It is often represented using a knowledge graph (KG) that tracks key entities, events, and relationships.', additional_kwargs={}, response_metadata={})]

In [32]:
second_question = "Give an example to understand it."
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

For example, a 360-degree view of a customer "Acma Retail Inc" would include not only the company's internal information, such as its account history and transactions, but also external information like news reports of its bankruptcy filing, its supply chain relationships, and potential financial stress on its suppliers. This integrated view helps financial institutions understand the customer's situation more thoroughly.


In [33]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [34]:
store={}

In [35]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [36]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [37]:
conversational_rag_chain.invoke(
    {"input": "what is a 360 degree view??"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'A 360-degree view refers to a comprehensive and integrated view of a customer, combining external information with internal company data. It provides a complete picture of a customer by linking various data sources, such as financial news, supply chain relationships, and internal customer information. This integrated view helps businesses better manage customer relationships and identify potential risks or opportunities.'

In [38]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is a 360 degree view??', additional_kwargs={}, response_metadata={}), AIMessage(content='A 360-degree view refers to a comprehensive and integrated view of a customer, combining external information with internal company data. It provides a complete picture of a customer by linking various data sources, such as financial news, supply chain relationships, and internal customer information. This integrated view helps businesses better manage customer relationships and identify potential risks or opportunities.', additional_kwargs={}, response_metadata={})])}

In [39]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways to create a 360-degree view include integrating external data sources, such as financial news and commercially sourced data, with internal company information. This is often achieved through data integration processes, including loading data into a knowledge graph engine, and using a schema to define key entities, events, and relationships.'

In [40]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is a 360 degree view??', additional_kwargs={}, response_metadata={}), AIMessage(content='A 360-degree view refers to a comprehensive and integrated view of a customer, combining external information with internal company data. It provides a complete picture of a customer by linking various data sources, such as financial news, supply chain relationships, and internal customer information. This integrated view helps businesses better manage customer relationships and identify potential risks or opportunities.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Common ways to create a 360-degree view include integrating external data sources, such as financial news and commercially sourced data, with internal company information. This is often achieved through data integration processes, including l

In [41]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")


User: what is a 360 degree view??

AI: A 360-degree view refers to a comprehensive and integrated view of a customer, combining external information with internal company data. It provides a complete picture of a customer by linking various data sources, such as financial news, supply chain relationships, and internal customer information. This integrated view helps businesses better manage customer relationships and identify potential risks or opportunities.

User: What are common ways of doing it?

AI: Common ways to create a 360-degree view include integrating external data sources, such as financial news and commercially sourced data, with internal company information. This is often achieved through data integration processes, including loading data into a knowledge graph engine, and using a schema to define key entities, events, and relationships.

